In [ ]:
!pip install langchain_community
!pip install pypdf
!pip install qdrant-client
!pip install sentence-transformers
!pip install gradio

In [2]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed
set_seed(1234)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
df = pd.read_csv('/content/train.csv')

In [4]:
ques_data = df['Question'].tolist()[:100]
answer_data = df['Answer'].tolist()[:100]

In [5]:
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
vectors = model.encode(ques_data)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams

client = QdrantClient(":memory:")

In [7]:
client.recreate_collection(
    collection_name="doc_data",
    vectors_config=VectorParams(size=len(vectors[0]), distance=Distance.COSINE),
)

<ipython-input-7-62cfb2444473>:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [8]:
client.upload_collection(
    collection_name="doc_data",
    ids=[i for i in range(len(ques_data))],
    vectors=vectors,
)

In [9]:
def fetch_best_match(question):
  model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
  ques_vector = model.encode(question)
  result = client.query_points(collection_name="doc_data",query=ques_vector)
  sim_ids = []
  for i in result.points:
    sim_ids.append(i.id)
  context = answer_data[sim_ids[0]]
  return context

In [11]:
def fetch_llm_response(question,context):
  model_checkpoint = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
  tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
  model = AutoModelForCausalLM.from_pretrained(model_checkpoint, device_map="cuda")
  chat = [
    {"role": "user", "content": f"this is question {question} asked by user you are a chatbot answer the question based on this context {context} in not more than 3-4 points"}
  ]
  token_inputs = tokenizer.apply_chat_template(chat, tokenize=True, return_tensors="pt", add_generation_prompt=True).to("cuda")
  token_outputs = model.generate(input_ids=token_inputs, do_sample=True, max_new_tokens=500, temperature=.5)
  new_tokens = token_outputs[0][token_inputs.shape[-1]:]
  decoded_output = tokenizer.decode(new_tokens, skip_special_tokens=True)
  return decoded_output

In [15]:
import gradio as gr

# Function to handle chatbot responses
def chatbot_response(user_input):
    if user_input:
      context = fetch_best_match(user_input)
      return fetch_llm_response(user_input,context)
    else:
        return "Bot: Please enter a message."

# Gradio interface
iface = gr.Interface(
    fn=chatbot_response,
    inputs="text",
    outputs="text",
    title="Medical RAG",
    description="Type your issue below and the bot will respond."
)

# Launch the Gradio interface
iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://8ac0a25881b4d872ad.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
